## Trabajo Práctico: Red Neuronal Lineal.

aca hay que poner una introducción supongo idk

In [1]:
import numpy as np
import sys

from alc import *
# TODO esto todo va a estar en alc cuando terminemos y hay que borrarlo
sys.path.append(".")
sys.path.append("./src")
sys.path.append("../src")
from labo00_auxiliares import *
from labo01_errores_igualdad import *
from labo02_TLs_basicas import *
from labo03_normas import *
from labo04_LU import *
from labo05_QR import *
from labo06_AVs import *
from labo07_markov import *
from labo08_SVD import *
import os
os.getcwd()[-3:]



'/TP'

no se explicar que hacemos las pruebas

Recordamos, los "t" son los valores de entrenamiento, y los v son los de testeo

In [2]:
Xt, Yt, Xv, Yv = cargarDatasetReducido("cats_and_dogs", 5, 3)
print(Xt, Xt.shape)
print(Xv, Xv.shape)
print(Yt, Yt.shape)
print(Yv, Yv.shape)

[[-0.03650029 -0.12246419  0.58658326 -0.0542641   0.11602265  0.25493175]
 [ 0.11822625  0.26049453 -0.02440555  0.00112447 -0.02789933  0.11979685]
 [ 0.09710626 -0.00973467  0.19654617  0.2523477   0.25783956 -0.11589213]
 [ 0.12151048  0.42283142  0.32398704  0.06658641  0.03125512  0.33627212]
 [ 0.00340588  0.02018487  0.7119048   0.05974435 -0.06707007  0.50322443]] (5, 6)
[[ 0.01444551  0.14617081  0.35378024  0.23573318  0.13566768 -0.06975192]
 [ 0.31209746  0.7037203  -0.01706209 -0.09351135  0.30183068  0.31628153]
 [-0.07121716  0.32912827  0.07275005  0.12642002  0.21498148 -0.06499534]
 [ 0.76810056  0.7381562   0.02793839 -0.03192531  0.521346    0.44776803]
 [ 0.20623364  0.19037941  0.00360998  0.02304114  0.4993087  -0.00829356]] (5, 6)
[[0 0 0 1 1 1]
 [1 1 1 0 0 0]] (2, 6)
[[0 0 0 1 1 1]
 [1 1 1 0 0 0]] (2, 6)


PinSVD

In [ ]:
def pinvSVD(X, Y):
    """ 
    Obtiene los pesos utilizando la Descomposición
    en Valores Singulares para la resolución de la pseudo-inversa usando el algoritmo 2. 
    U: Matriz de autovectores por izquierda de SVD
    S: Matriz Sigma de valores singulares
    V: Matriz de autovectores por derecha de SVD
    Y: matriz de targets de entrenamiento. 
    retorna pesos W
    """

    U,S,V = svd_completa(X)
    S_ALaMenosUno = np.zeros((len(S),len(S[0])))
    for i in len(S):
        if S[i][i] >0:               # si uno es cero, los que le siguen tambien
            S_ALaMenosUno[i][i] = (S[i][i])**(-1)
    SigmaMas = traspuesta(S_ALaMenosUno[:,len(S)])
    V1 = V[:,len(S)]
    W = matmul(Y, matmul((V1, SigmaMas),traspuesta(U)))

    return W

### QR
voy a intentar que funcione con QR que es la única que creo que ya esta implementada. 

problema: solo funciona sobre matrices cuadradas, hay que cambiar eso
problema: muy lento. vamos a cachearlo una vez calculado en un archivo para no tener que volver a hacerlo mientras testeamos cosas


In [3]:
def cache(calc, name):
    cache_save_enabled = False
    if os.path.isfile(f"{name}.npy"):
        return np.load(f"{name}.npy")
    else:
        val = calc()
        if cache_save_enabled:
            np.save(f"{name}.npy", val)
        return val
    

#### Por Gram shmidt

In [4]:
Q, R = cache(lambda: QR_con_GS(traspuesta(Xt)), "QRGS")

assert np.allclose(Q @ R, traspuesta(Xt)) # lo hago con @ solo aca para probar

In [5]:
print(f"Q: {np.round(Q,3).shape}")
print(np.round(Q,3))
print(f"R: {np.round(R,3).shape}")
print(np.round(R,3))

W = cache(lambda:pinvGramSchmidt(Q,R,Yt), "WGS")

print(f"W: {np.round(W,3).shape}")

YvEsperados = cache(lambda:matmul(W, Xv), "YvEsperadosGS")
print(YvEsperados)
print(Yv)




Q: (6, 5)
[[-0.055  0.375  0.291 -0.766  0.397]
 [-0.184  0.818  0.117  0.313 -0.369]
 [ 0.882  0.021  0.159  0.097 -0.006]
 [-0.082 -0.006  0.645  0.495  0.549]
 [ 0.175 -0.07   0.559 -0.243 -0.575]
 [ 0.384  0.429 -0.385  0.033  0.271]]
R: (5, 5)
[[ 0.665 -0.035  0.15   0.33   0.801]
 [ 0.     0.31  -0.037  0.54   0.253]
 [ 0.     0.     0.41   0.067 -0.076]
 [ 0.     0.     0.     0.107  0.135]
 [ 0.     0.     0.     0.     0.198]]
W: (2, 5)
[[ 0.44111843  0.37165913 -0.44966705 -0.04391745  0.91671111 -0.00916365]
 [ 0.41078376  5.58604604  0.8083613   0.04247648  2.07319364  1.32419963]]
[[0 0 0 1 1 1]
 [1 1 1 0 0 0]]


#### Por HH

In [6]:
QHH, RHH = cache(lambda: QR_con_HH(traspuesta(Xt)), "QRHH")

assert np.allclose(QHH @ RHH, traspuesta(Xt)) # lo hago con @ solo aca para probar

In [7]:
print(f"Q: {np.round(QHH,3).shape}")
print(np.round(QHH,3))
print(f"R: {np.round(RHH,3).shape}")
print(np.round(RHH,3))

WHH = cache(lambda:pinvHouseHolder(QHH,RHH,Yt), "WHH")

print(f"W: {np.round(WHH,3).shape}")
YvEsperadosHH = cache(lambda:matmul(WHH, Xv), "YvEsperadosHH")
print(YvEsperadosHH)
print(Yv)




Q: (6, 5)
[[-0.055 -0.375 -0.291  0.766  0.397]
 [-0.184 -0.818 -0.117 -0.313 -0.369]
 [ 0.882 -0.021 -0.159 -0.097 -0.006]
 [-0.082  0.006 -0.645 -0.495  0.549]
 [ 0.175  0.07  -0.559  0.243 -0.575]
 [ 0.384 -0.429  0.385 -0.033  0.271]]
R: (5, 5)
[[ 0.665 -0.035  0.15   0.33   0.801]
 [-0.    -0.31   0.037 -0.54  -0.253]
 [-0.    -0.    -0.41  -0.067  0.076]
 [ 0.    -0.    -0.    -0.107 -0.135]
 [ 0.    -0.     0.    -0.     0.198]]
W: (2, 5)
[[ 0.44111843  0.37165913 -0.44966705 -0.04391745  0.91671111 -0.00916365]
 [ 0.41078376  5.58604604  0.8083613   0.04247648  2.07319364  1.32419963]]
[[0 0 0 1 1 1]
 [1 1 1 0 0 0]]
